In [2]:
# Load WSC dataset

import xml.etree.ElementTree as etree
import json
import numpy as np

import logging
import numpy
import os


def softmax(x):
    return np.exp(x)/sum(np.exp(x))

In [3]:
tree = etree.parse('WSCollection.xml')
root = tree.getroot()
original_problems = root.getchildren()
problems = list()

for original_problem in original_problems:
    problem = dict()
    for information in original_problem.getchildren():
        if information.tag == 'answers':
            answers = information.getchildren()
            answer_list = list()
            for answer in answers:
                answer_list.append(answer.text.strip())
            problem['answers'] = answer_list
        elif information.tag == 'text':
            texts = information.getchildren()
            text_dict = dict()
            for text1 in texts:
                text_dict[text1.tag] = text1.text.replace('\n', ' ').strip()
            problem['text'] = text_dict
        elif information.tag == 'quote':
            pass
        else:
            problem[information.tag] = information.text.replace(' ', '')
    problems.append(problem)

print(problems[0])

all_sentences = list()
for question in problems:
    sentence = question['text']['txt1'] + ' ' + question['text']['pron'] + ' ' + question['text']['txt2']
    all_sentences.append(sentence)
    # print(sentence)
    

{'text': {'txt1': 'The city councilmen refused the demonstrators a permit because', 'pron': 'they', 'txt2': 'feared violence.'}, 'answers': ['The city councilmen', 'The demonstrators'], 'correctAnswer': 'A', 'source': '(Winograd1972)'}


/home/xzhaoar/anaconda3/envs/gpt2/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/xzhaoar/anaconda3/envs/gpt2/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  
/home/xzhaoar/anaconda3/envs/gpt2/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  app.launch_new_instance()
/home/xzhaoar/anaconda3/envs/gpt2/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  # Remove the CWD from sys.path while we load stuff.


In [4]:
import json
import numpy as np
import tensorflow as tf
import model, sample, encoder



model_name = '774M'
models_dir = '../models'

enc = encoder.get_encoder(model_name, models_dir)

batch_size = 1
seed=None
nsamples=1


hparams = model.default_hparams()
with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
    hparams.override_from_dict(json.load(f))
    
length = hparams.n_ctx // 2

answer_collector = []


def logits_score(logits,skeleton_tokens, context_tokens):
    score = 1
    start_index = len(skeleton_tokens) - 1 
    end_index = len(context_tokens) - 1
    
    for i in range(end_index - start_index): 
        m = softmax(logits[start_index+i])
        score *= m[context_tokens[start_index+i+1]]
    
    return score
        


with tf.Session(graph=tf.Graph()) as sess:
    
    context = tf.placeholder(tf.int32, [batch_size, None])
    np.random.seed(seed)
    tf.set_random_seed(seed)
    
    context_tokens = []

    output = model.model(hparams=hparams, X=context, past=None, reuse=tf.AUTO_REUSE)

    saver = tf.train.Saver()
    ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
    saver.restore(sess, ckpt)

    for i in range(273):      
        
        if problems[i]['text']['txt1'] != ".":
            ans0 = problems[i]['answers'][0].replace("The","the")
            ans1 = problems[i]['answers'][1].replace("The","the")
        else:
            ans0 = problems[i]['answers'][0]
            ans1 = problems[i]['answers'][1]
        
        skeleton1 = problems[i]['text']['txt1'] + ' ' + problems[i]['answers'][0]
        skeleton2 = problems[i]['text']['txt1'] + ' ' + problems[i]['answers'][1]
        raw_text1 = problems[i]['text']['txt1'] + ' ' + problems[i]['answers'][0]  + ' ' + problems[i]['text']['txt2']
        raw_text2 = problems[i]['text']['txt1'] + ' ' + problems[i]['answers'][1] + ' ' +  problems[i]['text']['txt2']
        context_tokens1 = enc.encode(raw_text1)
        context_tokens2 = enc.encode(raw_text2)
        skeleton_tokens1 = enc.encode(skeleton1)
        skeleton_tokens2 = enc.encode(skeleton2)
        
        out1 = sess.run(output, feed_dict={context: [context_tokens1 for _ in range(batch_size)]})
        out2 = sess.run(output, feed_dict={context: [context_tokens2 for _ in range(batch_size)]})
        
        logits1 = out1['logits'][:, :, :hparams.n_vocab]
        logits2 = out2['logits'][:, :, :hparams.n_vocab]
            
        score1 = logits_score(logits1[0],skeleton_tokens1,context_tokens1)
        score2 = logits_score(logits2[0],skeleton_tokens2,context_tokens2)  
        
        correctAnswer = problems[i]["correctAnswer"]

        if score1 >= score2:
            predictedAnswer = "A"
        else:
            predictedAnswer = "B"
        # A. Problem
        answer_collector.append(predictedAnswer in correctAnswer)

W0730 18:08:22.649106 140254510032704 deprecation_wrapper.py:119] From /home/xzhaoar/gpt-2-master/src/model.py:148: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0730 18:08:22.661986 140254510032704 deprecation_wrapper.py:119] From /home/xzhaoar/gpt-2-master/src/model.py:152: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0730 18:08:22.717363 140254510032704 deprecation_wrapper.py:119] From /home/xzhaoar/gpt-2-master/src/model.py:36: The name tf.rsqrt is deprecated. Please use tf.math.rsqrt instead.

W0730 18:08:29.380876 140254510032704 deprecation.py:323] From /home/xzhaoar/anaconda3/envs/gpt2/lib/python3.7/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [5]:
print(len(answer_collector))
print(np.sum(answer_collector)/273)

273
0.6923076923076923
